In [199]:
import warnings

import numpy as np
import pandas as pd
import pingouin as pg
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
from scipy.stats import shapiro, levene
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [200]:
df = pd.read_csv('source/FReDA4.csv')
df2 = pd.read_csv('source/FReDA3.csv')
df["Group4"] = None

In [201]:
# couples_satisfied = df[df["Group3"] == "Couple Satisfaction"].copy()
# couples_deprived = df[df["Group3"] == "Couple Deprivation"].copy()
# couples_saturated = df[df["Group3"] == "Couple Saturation"].copy()
# couples_mixed = df[df["Group3"] == "Couple Mixed"].copy()

In [202]:
# Satisfied_couples = df[df["Group3"] == "Couple Satisfaction"].copy()
# deprived_couples = df[df["Group2"] == "Couple Deprivation"].copy()
# deprived_one = df[df["Group2"] == "One-sided Deprivation"].copy()
# deprived_me = df[df["Group1"] == "SubGroup2"].copy()
# deprived_partner = df[df["Group1"] == "SubGroup7"].copy()
# saturated_couples = df[df["Group2"] == "Couple Saturation"].copy()
# saturated_one = df[df["Group2"] == "One-sided Saturation"].copy()
# saturated_me = df[df["Group1"] == "SubGroup5"].copy()
# saturated_partner = df[df["Group1"] == "SubGroup8"].copy()

In [203]:
# Satisfied
df.loc[df['Group1'] == 'SubGroup1', 'Group4'] = 'Satisfied'

# Deprived groups
df.loc[df['Group1'] == 'SubGroup3', 'Group4'] = 'Deprived_Couples'
df.loc[df['Group2'] == 'One-sided Deprivation', 'Group4'] = 'Deprived_One'
df.loc[df['Group1'] == 'SubGroup2', 'Group4'] = 'Deprived_Me'
df.loc[df['Group1'] == 'SubGroup7', 'Group4'] = 'Deprived_Partner'

# Saturated groups
df.loc[df['Group1'] == 'SubGroup6', 'Group4'] = 'Saturated_Couples'
df.loc[df['Group2'] == 'One-sided Saturation', 'Group4'] = 'Saturated_One'
df.loc[df['Group1'] == 'SubGroup5', 'Group4'] = 'Saturated_Me'
df.loc[df['Group1'] == 'SubGroup8', 'Group4'] = 'Saturated_Partner'

# Mixed groups
df.loc[df['Group1'] == 'SubGroup4', 'Group4'] = 'Mixed_Couples'
df.loc[df['Group1'] == 'SubGroup9', 'Group4'] = 'Mixed_Couples'

In [204]:
print(df['Group4'].isna().sum())

0


In [205]:
df = df.rename(columns={
    'Self-esteem': 'Self_esteem',
    'Life Satisfaction': 'Life_Satisfaction',
    'Communication Quality': 'Communication_Quality',
    'Relationship Satisfaction': 'Relationship_Satisfaction',
    'Conflict Management': 'Conflict_Management'
})

In [206]:
# traits = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']
# traits = ['Depressiveness', 'Loneliness', 'Self_esteem', 'Life_Satisfaction', 'Health']
traits = ["Communication_Quality", "Relationship_Satisfaction", "Conflict_Management"]

In [207]:
df = df.dropna(subset=traits).copy()

In [208]:
# -----------------------
# Assumption checks
# -----------------------
for trait in traits:
    print(f"\n--- {trait} ---")

    # Shapiro normality per group
    for group in df['Group4'].unique():
        data = df[df['Group4'] == group][trait]
        if len(data) >= 3:  # Shapiro requires >=3
            stat, p = shapiro(data)
            # print(f"{group} Shapiro p={p:.3f}")

    # Levene test for homogeneity of variance
    groups_data = [df[df['Group4'] == g][trait] for g in df['Group4'].unique()]
    stat, p = levene(*groups_data)
    print(f"Levene test p={p:.3f} (p>0.05 = equal variances)")


--- Communication_Quality ---
Levene test p=0.000 (p>0.05 = equal variances)

--- Relationship_Satisfaction ---
Levene test p=0.000 (p>0.05 = equal variances)

--- Conflict_Management ---
Levene test p=0.000 (p>0.05 = equal variances)


In [209]:
# -----------------------
# ANOVA + Effect size
# -----------------------
for trait in traits:
    print(f"\n=== ANOVA for {trait} ===")

    # OLS model
    model = ols(f'{trait} ~ C(Group4)', data=df).fit()

    # Standard ANOVA table
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table)

    # Eta squared from pingouin
    eta2 = pg.anova(dv=trait, between='Group4', data=df,  effsize='n2', detailed=True)
    print(f"Eta squared = {eta2.loc[0, 'n2']:.3f}")


=== ANOVA for Communication_Quality ===
                  sum_sq       df           F         PR(>F)
C(Group4)   13081.689283      7.0  170.783463  4.991319e-242
Residual   133149.388246  12168.0         NaN            NaN
Eta squared = 0.089

=== ANOVA for Relationship_Satisfaction ===
                 sum_sq       df           F         PR(>F)
C(Group4)   2538.978400      7.0  163.865291  1.777567e-232
Residual   26933.524885  12168.0         NaN            NaN
Eta squared = 0.086

=== ANOVA for Conflict_Management ===
                  sum_sq       df          F         PR(>F)
C(Group4)    4797.995071      7.0  72.687068  1.634353e-103
Residual   114742.367281  12168.0        NaN            NaN
Eta squared = 0.040


In [210]:
# -----------------------
# Post-hoc tests
# -----------------------
for trait in traits:
    print(f"\n--- Post-hoc comparisons for {trait} ---")

    # Tukey HSD
    tukey = pairwise_tukeyhsd(endog=df[trait], groups=df['Group4'], alpha=0.05)
    print(tukey)


--- Post-hoc comparisons for Communication_Quality ---
           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2      meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------
 Deprived_Couples       Deprived_Me   1.1975    0.0  0.9046  1.4904   True
 Deprived_Couples  Deprived_Partner   1.1086    0.0  0.8406  1.3767   True
 Deprived_Couples     Mixed_Couples   0.4752 0.0339  0.0194   0.931   True
 Deprived_Couples         Satisfied   2.7178    0.0  2.4686   2.967   True
 Deprived_Couples Saturated_Couples   2.3591    0.0  1.1947  3.5234   True
 Deprived_Couples      Saturated_Me   2.0624    0.0  1.4005  2.7242   True
 Deprived_Couples Saturated_Partner   2.2297    0.0  1.6544   2.805   True
      Deprived_Me  Deprived_Partner  -0.0889 0.9874 -0.3931  0.2153  False
      Deprived_Me     Mixed_Couples  -0.7223 0.0001 -1.2002 -0.2444   True
      Deprived_Me         Satisfied   1.5203